In [1]:
import os
from PIL import Image
import numpy as np
import torch
from transformers import ViTFeatureExtractor, ViTModel
from tqdm import tqdm

/opt/anaconda3/envs/SMCA/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
log_mel_spectrograms_path = '/Users/chy/Projects/Thesis/spectrograms'
mfcc_path = '/Users/chy/Projects/Thesis/mfcc'
output_path = '/Users/chy/Projects/Thesis/audio_extracted'

logmel_output_path = os.path.join(output_path, 'logmel_extracted')
mfcc_output_path = os.path.join(output_path, 'mfcc_extracted')

In [3]:
def verify_paths(paths):
    for path in paths:
        if not os.path.exists(path):
            raise FileNotFoundError(f"The path {path} does not exist or is not accessible.")

verify_paths([log_mel_spectrograms_path, mfcc_path, output_path])

In [4]:
os.makedirs(logmel_output_path, exist_ok=True)
os.makedirs(mfcc_output_path, exist_ok=True)

In [5]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

/opt/anaconda3/envs/SMCA/lib/python3.12/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [6]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)
model.eval()

ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTSdpaAttention(
          (attention): ViTSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUAct

In [7]:
def process_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=image, return_tensors="pt")
    inputs = inputs.to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    print(f"Processed {image_path} with feature shape: {last_hidden_states.shape}")
    return last_hidden_states.cpu().numpy()

In [8]:
def extract_features(input_path, output_path, feature_suffix=""):
    image_files = [f for f in os.listdir(input_path) if f.endswith(".png") or f.endswith(".jpg")]
    
    for filename in tqdm(image_files):
        image_path = os.path.join(input_path, filename)
        feature_vector = process_image(image_path)
        base_filename = os.path.splitext(filename)[0]
        output_file = os.path.join(output_path, f"feature_{base_filename}{feature_suffix}.npy")

        np.save(output_file, feature_vector)


In [11]:
extract_features(log_mel_spectrograms_path, logmel_output_path)
extract_features(mfcc_path, mfcc_output_path)

  0%|          | 1/1356 [00:00<06:01,  3.75it/s]

Processed /Users/chy/Projects/Thesis/spectrograms/tt5512872.png with feature shape: torch.Size([1, 197, 768])
Processed /Users/chy/Projects/Thesis/spectrograms/tt0180093.png with feature shape: torch.Size([1, 197, 768])


  0%|          | 5/1356 [00:00<02:10, 10.37it/s]

Processed /Users/chy/Projects/Thesis/spectrograms/tt0382628.png with feature shape: torch.Size([1, 197, 768])
Processed /Users/chy/Projects/Thesis/spectrograms/tt1299653.png with feature shape: torch.Size([1, 197, 768])
Processed /Users/chy/Projects/Thesis/spectrograms/tt0113189.png with feature shape: torch.Size([1, 197, 768])


  1%|          | 7/1356 [00:00<02:01, 11.08it/s]

...

Processed /Users/chy/Projects/Thesis/mfcc/tt0120619_mfcc.png with feature shape: torch.Size([1, 197, 768])
Processed /Users/chy/Projects/Thesis/mfcc/tt0896798_mfcc.png with feature shape: torch.Size([1, 197, 768])
Processed /Users/chy/Projects/Thesis/mfcc/tt0381849_mfcc.png with feature shape: torch.Size([1, 197, 768])


100%|█████████▉| 1354/1356 [02:01<00:00,  9.11it/s]

Processed /Users/chy/Projects/Thesis/mfcc/tt1567233_mfcc.png with feature shape: torch.Size([1, 197, 768])
Processed /Users/chy/Projects/Thesis/mfcc/tt0388500_mfcc.png with feature shape: torch.Size([1, 197, 768])
Processed /Users/chy/Projects/Thesis/mfcc/tt0299930_mfcc.png with feature shape: torch.Size([1, 197, 768])


100%|██████████| 1356/1356 [02:01<00:00, 11.16it/s]

Processed /Users/chy/Projects/Thesis/mfcc/tt0119906_mfcc.png with feature shape: torch.Size([1, 197, 768])
Processed /Users/chy/Projects/Thesis/mfcc/tt3526408_mfcc.png with feature shape: torch.Size([1, 197, 768])
